In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from dotenv import dotenv_values
# import time
import os

In [ ]:
options = webdriver.FirefoxOptions()
options.add_argument("-detach")
driver = webdriver.Firefox(options=options)

In [ ]:
BASE_URL = "https://course.testpad.chitkara.edu.in"
COURSE_NAME = "24CSE0208-Data Structures using Object Oriented Programming-2024-CSE-3Sem"

os.makedirs(COURSE_NAME, exist_ok = True)

In [ ]:
driver.get(BASE_URL)

wait = WebDriverWait(driver, 10)
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "loginIframe")))

In [ ]:
testpad_credentials = dotenv_values(".env")

email_element = driver.find_element(By.ID, "email")
email_element.send_keys(testpad_credentials["TESTPAD_EMAIL"])

password_element = driver.find_element(By.ID, "password")
password_element.send_keys(testpad_credentials["TESTPAD_PASSWORD"])

submit_button = driver.find_element(By.ID, "submit")
submit_button.click()

driver.switch_to.parent_frame()

In [ ]:
def wait_for_loader_disappearance(): wait.until(EC.invisibility_of_element_located(
    (By.ID, "loader-container")))

wait.until(EC.visibility_of_all_elements_located(
    (By.CLASS_NAME, "learning-module-card")))

learning_module_card = driver.find_element(
    By.XPATH, f"//a[text()='{COURSE_NAME}']").find_element(By.XPATH, "../..")

learning_module_href = learning_module_card.find_element(
    By.CSS_SELECTOR,
    "[class='icons-link-container tooltip-up'][data-customtooltip='Learning Content']"
)

wait_for_loader_disappearance()
wait.until(EC.element_to_be_clickable(learning_module_href)).click()

In [ ]:
"""
Wait for subtopics to show up
"""
wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "course-topics")))

course_topics = driver.find_elements(
    By.CLASS_NAME, "course-topics")

no_of_topics = len(course_topics)
if no_of_topics == 0:
    raise ValueError("No course topic showing!")

print(f"Number of subtopics- {no_of_topics}")

wait_for_loader_disappearance()

original_window = driver.current_window_handle

In [ ]:
EXTENSION_MAPPING = {
    'C': ".c",
    "Java": ".java",
    "C++": ".cpp",
    "Python": ".py",
    "SQL": ".sql"
}


def copy_coding_answer(answer_folder_path, question_number, question_name):
    # print(answer_folder_path)

    nav_attempts_tab = wait.until(
        EC.element_to_be_clickable((By.ID, "nav-attempts-tab")))
    wait_for_loader_disappearance()
    nav_attempts_tab.click()

    successes = wait.until(EC.visibility_of_all_elements_located(
        (By.CLASS_NAME, "success")))

    attempts_window = driver.current_window_handle
    successes[0].find_element(By.CSS_SELECTOR, "a.res").click()
    wait.until(EC.number_of_windows_to_be(3))

    for window in driver.window_handles:
        if window != original_window and window != attempts_window:
            driver.switch_to.window(window)
            break

    wait.until(EC.visibility_of_element_located(
        (By.CSS_SELECTOR, ".CodeMirror.cm-s-default")))
    coding_question_content = driver.execute_script("const cm = document.getElementsByClassName('CodeMirror cm-s-default')[0].CodeMirror;" + "return cm.getValue();")

    coding_language = driver.find_elements(
        By.CLASS_NAME, "filter-option-inner-inner")[-1].text
    
    with open(os.path.join(answer_folder_path, question_number + ". " + question_name + EXTENSION_MAPPING[coding_language]), "w") as answer_file:
        answer_file.write(coding_question_content)

    driver.close()
    driver.switch_to.window(attempts_window)

In [ ]:
def copy_textarea_answer(answer_file_path):
    # print(answer_file_path)

    nav_attempts_tab = wait.until(
        EC.element_to_be_clickable((By.ID, "nav-attempts-tab")))
    wait_for_loader_disappearance()
    nav_attempts_tab.click()

    successes = wait.until(EC.visibility_of_all_elements_located(
        (By.CLASS_NAME, "success")))

    attempts_window = driver.current_window_handle
    successes[0].find_element(By.CSS_SELECTOR, "a.res").click()
    wait.until(EC.number_of_windows_to_be(3))

    for window in driver.window_handles:
         if window != original_window and window != attempts_window:
              driver.switch_to.window(window)
              break
    
    mq_answers = wait.until(EC.visibility_of_any_elements_located(
        (By.CSS_SELECTOR, ".test_inputs.input-option")))
    mq_answers = [item for item in mq_answers if item.is_displayed()]

    with open(os.path.join(answer_file_path, "Answer.txt"), "w") as answer_file:      
        for i, mq_answer in enumerate(mq_answers):
            answer_file.write(mq_answer.text + ("\n\n\n" if i != len(mq_answers) - 1 else ""))

    driver.close()
    driver.switch_to.window(attempts_window)

In [ ]:
def copy_mcq_answer(answer_file_path):
    # print(answer_file_path)

    nav_attempts_tab = wait.until(
        EC.element_to_be_clickable((By.ID, "nav-attempts-tab")))
    wait_for_loader_disappearance()
    nav_attempts_tab.click()

    successes = wait.until(EC.visibility_of_all_elements_located(
        (By.CLASS_NAME, "success")))

    attempts_window = driver.current_window_handle
    successes[0].find_element(By.CSS_SELECTOR, "a.res").click()
    wait.until(EC.number_of_windows_to_be(3))

    for window in driver.window_handles:
        if window != original_window and window != attempts_window:
            driver.switch_to.window(window)
            break

    mcq_options = wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, ".col-12.mt-2.div-radio-btn-choose-answer")))
    wait.until(EC.visibility_of_any_elements_located((By.CLASS_NAME, "dmr")))
    correct_answer = 0
    for i in range(len(mcq_options)):
        mcq_option = mcq_options[i]
        if mcq_option.find_element(By.CLASS_NAME, "dmr").is_displayed():
            correct_answer = str(i + 1)
            break
    else:
        raise ValueError("Correct answer not found! ❌")
    
    with open(os.path.join(answer_file_path, "Answer.txt"), "w") as answer_file:
        answer_file.write(correct_answer)

    driver.close()
    driver.switch_to.window(attempts_window)

In [ ]:
def extract_subtopic_answers(subtopic_number: str, subtopic_name: str, subtopic_questions: list):
    no_of_subtopic_questions = len(subtopic_questions)
    for i in range(no_of_subtopic_questions):
        question = subtopic_questions[i]

        question_number = question.find_element(
            By.CLASS_NAME, "course-question-number").text
        question_name = question.find_element(
            By.CLASS_NAME, "course-question-name").text
        question_type = question.find_element(
            By.CLASS_NAME, "course-question-type").text

        question_status = bool(question.find_elements(
            By.CSS_SELECTOR, ".question-complete-tick-mark-mv > img"))

        if question_status and question_type != "Tutorial":
            # if question_type == "MQ" or question_type == "MCQ":
                answer_file_path = os.path.join(COURSE_NAME, subtopic_number + ". " + subtopic_name.replace(os.sep, ""), question_number + ". " + question_name.replace(os.sep, ""))
                os.makedirs(answer_file_path, exist_ok=True)
                
                print(f"Copying solution to question- {question_name} ({question_type})")

                question.find_element(
                    By.CLASS_NAME, "isQuestionModal").click()

                wait.until(EC.number_of_windows_to_be(2))
                for window in driver.window_handles:
                    if window != original_window:
                        driver.switch_to.window(window)
                        break

                if question_type == "Coding":
                    copy_coding_answer(answer_file_path, question_number, question_name)
                    # break
                elif question_type == "MCQ":
                    copy_mcq_answer(answer_file_path)
                    # break
                elif question_type == "MQ":
                    copy_textarea_answer(answer_file_path)
                    # break
                else:
                    print(f"Unknown question type- {question_type}")
                    break
                print("Succesfully copied solution✅\n")

                # time.sleep(1)  # For the purpose of visual testing
                driver.close()
                driver.switch_to.window(original_window)

    # print("Subtopic questions execution complete.")

In [ ]:
"""
# Print course topics

for course_topic in course_topics:
    course_topic_number = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-number > p").text
    course_topic_name = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-name > p").text
    print(course_topic_number, course_topic_name)
"""

driver.switch_to.window(original_window)

course_topics = driver.find_elements(
    By.CLASS_NAME, "course-topics")
wait_for_loader_disappearance()
# time.sleep(2) # For the purpose of visual testing

# t = 2
# for i in range(t-1, t):
for i in range(len(course_topics)):
    course_topic = course_topics[i]

    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", course_topic)

    subtopic_number = course_topic.find_element(
        By.CSS_SELECTOR, "div.course-topics-number > p").text
    


    """Code to determine whether or not to click on subtopic. Click if not already clicked."""
    course_topics_item = driver.find_elements(
        By.CLASS_NAME, "course-topics")
    preclicked_subtopic = -1
    for j in range(len(course_topics_item)):
        item = course_topics_item[j]
        if "clicked" in item.get_attribute("class"):
            preclicked_subtopic = j
            break

    if not i == preclicked_subtopic:
        wait_for_loader_disappearance()
        course_topic.click()
        # print(f"Subtopic number {subtopic_number} clicked.")
    """"""




    wait_for_loader_disappearance()
    subtopic_name = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-name > p").text

    subtopic_questions = driver.find_elements(By.CLASS_NAME, "course-question")
    subtopic_questions = [item for item in subtopic_questions if item.is_displayed()]
    if len(subtopic_questions) == 0:
        raise ValueError("No subtopic questions! ❌")

    print("#" * 30)
    print(f"{subtopic_number}. {subtopic_name}, ({len(subtopic_questions)} questions)")

    no_of_incomplete_questions = 0
    for question in subtopic_questions:
        if not question.find_elements(By.CSS_SELECTOR, "div.question-complete-tick-mark-mv > img"):
            no_of_incomplete_questions += 1
    print(f"{no_of_incomplete_questions}/{len(subtopic_questions)} incomplete questions ❓")
    print("#" * 30, "\n\n")

    extract_subtopic_answers(
    subtopic_number, subtopic_name, subtopic_questions)